In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import speech_recognition as sr
import pyttsx3

# Load the dataset
df = pd.read_csv("DiseaseAndSymptoms.csv")

# Split the data into features (X) and target (y)
X = df.drop('Disease', axis=1)  # Features
y = df['Disease']  # Target

# Encode categorical variables using LabelEncoder
label_encoders = {}
for col in X.columns:
    if X[col].dtype == 'object':
        label_encoders[col] = LabelEncoder()
        X[col] = label_encoders[col].fit_transform(X[col])

# One-hot encode categorical variables
one_hot_enc = OneHotEncoder(handle_unknown='ignore')
X_encoded = one_hot_enc.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# # Initialize the decision tree classifier
# clf_decision_tree = DecisionTreeClassifier()
# clf_decision_tree.fit(X_train, y_train)

# Initialize the random forest classifier
clf_random_forest = RandomForestClassifier()
clf_random_forest.fit(X_train, y_train)

# # Predictions
# y_pred_decision_tree = clf_decision_tree.predict(X_test)
# y_pred_random_forest = clf_random_forest.predict(X_test)

# Calculate accuracy
accuracy_decision_tree = (y_test == y_pred_decision_tree).mean()
accuracy_random_forest = (y_test == y_pred_random_forest).mean()

# Initialize the speech recognition engine
recognizer = sr.Recognizer()

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Function to speak the response
def speak(text):
    print("Assistant:", text)
    engine.say(text)
    engine.runAndWait()

# # Function to predict disease based on symptoms using Decision Tree
# def predict_disease_decision_tree(symptoms_input):
#     input_data = np.zeros((1, len(X.columns)), dtype=int)
#     for symptom in symptoms_input:
#         if symptom in label_encoders:
#             input_data[0, X.columns.get_loc(symptom)] = label_encoders[symptom].transform([symptom])[0]
#     input_data_encoded = one_hot_enc.transform(input_data)
#     prediction = clf_decision_tree.predict(input_data_encoded)
#     return prediction[0]

# Function to predict disease based on symptoms using Random Forest
def predict_disease_random_forest(symptoms_input):
    input_data = np.zeros((1, len(X.columns)), dtype=int)
    for symptom in symptoms_input:
        if symptom in label_encoders:
            input_data[0, X.columns.get_loc(symptom)] = label_encoders[symptom].transform([symptom])[0]
    input_data_encoded = one_hot_enc.transform(input_data)
    prediction = clf_random_forest.predict(input_data_encoded)
    return prediction[0]

# Function to get symptoms from the user through voice
def get_symptoms_from_user_voice():
    symptoms_input = []
    while True:
        with sr.Microphone() as source:
            print("Listening for symptoms...")
            try:
                audio = recognizer.listen(source, timeout=5)  # Set a timeout of 5 seconds
                print("Recognizing...")
                user_input = recognizer.recognize_google(audio).lower()
                print("You said:", user_input)
                if user_input == "stop":
                    break  # Break out of the loop if user says "stop"
                else:
                    symptoms_input.append(user_input)
                    speak("Next")  # Acknowledge each symptom with "Next"
            except sr.WaitTimeoutError:
                speak("Sorry, I didn't hear anything. Can you please repeat?")  # Prompt user to repeat
                continue
            except sr.UnknownValueError:
                speak("Sorry, I couldn't understand what you said. Can you please repeat?")  # Prompt user to repeat
            except sr.RequestError as e:
                speak("Sorry, there was an error with the request. {0}".format(e))
            except TimeoutError:
                speak("Sorry, I didn't hear anything. Can you please repeat?")  # Prompt user to repeat
                continue
    return symptoms_input

# Function to ask the user if they are satisfied
def ask_satisfaction():
    while True:
        with sr.Microphone() as source:
            print("Are you satisfied with the prediction? (yes/no)")
            try:
                audio = recognizer.listen(source, timeout=5)  # Set a timeout of 5 seconds
                print("Recognizing...")
                user_input = recognizer.recognize_google(audio).lower()
                print("You said:", user_input)
                if user_input in ["yes", "no"]:
                    return user_input
                else:
                    speak("Please say 'yes' or 'no'.")
            except sr.UnknownValueError:
                speak("Sorry, I couldn't understand what you said. Can you please repeat?")  # Prompt user to repeat
            except sr.RequestError as e:
                speak("Sorry, there was an error with the request. {0}".format(e))
            except TimeoutError:
                speak("Sorry, I didn't hear anything. Can you please repeat?")  # Prompt user to repeat

# Main function to start the voice assistant
def main_voice():
    speak("Hello! I am your voice assistant. Let's predict your disease.")
    chat_history = []
    continue_running = True
    while continue_running:
        symptoms_input = get_symptoms_from_user_voice()
        if symptoms_input:
            predicted_disease_decision_tree = predict_disease_decision_tree(symptoms_input)
            predicted_disease_random_forest = predict_disease_random_forest(symptoms_input)
            
            # speak("Based on your symptoms, the predicted disease using Decision Tree is " + predicted_disease_decision_tree)
            speak("The predicted disease using Random Forest is " + predicted_disease_random_forest)
            
            satisfaction = ask_satisfaction()
            if satisfaction == "yes":
                speak("Thank you! Stay healthy.")
                continue_running = False
            else:
                speak("Redirecting you to the doctor's dashboard.")
                # Add code to redirect to the doctor's dashboard
                break
        else:
            speak("No symptoms provided. Can you please repeat?")
    # print("Chat History:")
    # for message in chat_history:
    #     print(message[0], message[1])

if __name__ == "__main__":
    main_voice()


Assistant: Hello! I am your voice assistant. Let's predict your disease.
Listening for symptoms...
Recognizing...
You said: headache
Assistant: Next
Listening for symptoms...
Assistant: Sorry, I didn't hear anything. Can you please repeat?
Listening for symptoms...
Recognizing...
Assistant: Sorry, I couldn't understand what you said. Can you please repeat?
Listening for symptoms...
Recognizing...
Assistant: Sorry, I couldn't understand what you said. Can you please repeat?
Listening for symptoms...
Recognizing...
You said: headache
Assistant: Next
Listening for symptoms...
Recognizing...
You said: fever
Assistant: Next
Listening for symptoms...
Recognizing...
You said: cold cold
Assistant: Next
Listening for symptoms...
Recognizing...
Assistant: Sorry, I couldn't understand what you said. Can you please repeat?
Listening for symptoms...
Recognizing...
Assistant: Sorry, I couldn't understand what you said. Can you please repeat?
Listening for symptoms...
Assistant: Sorry, I didn't hear 

C:\Users\Hp\miniconda3\envs\myenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\Hp\miniconda3\envs\myenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Assistant: The predicted disease using Random Forest is Common Cold
Are you satisfied with the prediction? (yes/no)
Recognizing...
Assistant: Sorry, I couldn't understand what you said. Can you please repeat?
Are you satisfied with the prediction? (yes/no)
Recognizing...
You said: no no
Assistant: Please say 'yes' or 'no'.
Are you satisfied with the prediction? (yes/no)
Recognizing...
Assistant: Sorry, I couldn't understand what you said. Can you please repeat?
Are you satisfied with the prediction? (yes/no)
Recognizing...
You said: no
Assistant: Redirecting you to the doctor's dashboard.
Chat History:
